In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import csv
import time
import os

# 크롬 드라이버 경로 설정
chromedriver_path = os.path.join(os.getcwd(), "chromedriver.exe")  # 크롬 드라이버 경로를 입력해주세요.

# 크롬 드라이버 서비스 객체 생성
service = Service(chromedriver_path)

# 웹 드라이버 객체 생성
driver = webdriver.Chrome(service=service)

# 크롤링할 URL
url = 'https://www.boannews.com/media/t_list.asp?mkind=0'

# 웹 페이지 열기
driver.get(url)

# 뉴스 데이터를 저장할 리스트
news_data = []

# 크롤링할 페이지 수
num_pages = 2  # 원하는 페이지 수를 입력해주세요.

for page in range(1, num_pages + 1):
    # 현재 페이지의 HTML 소스 코드 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 뉴스 목록 찾기
    news_list = soup.select('.news_list a:not(.news_content)')

    for news in news_list:
        # 뉴스 링크 추출
        news_url = 'https://www.boannews.com' + news['href']

        # 새 탭에서 뉴스 링크 열기
        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[-1])
        driver.get(news_url)

        # 뉴스 내용 크롤링
        news_html = driver.page_source
        news_soup = BeautifulSoup(news_html, 'html.parser')

        # 뉴스 제목
        news_title = news_soup.select_one('#news_title02 h1').text.strip()

        # 뉴스 날짜
        news_date = ':'.join(news_soup.select_one('#news_util01').text.strip().split(':')[1:]).strip()

        # 뉴스 내용
        news_content = news_soup.select_one('#news_content').text.strip()

        # 크롤링한 데이터를 리스트에 추가
        news_data.append([news_title, news_date, news_content])

        # 탭 닫기
        driver.close()

        # 원래 탭으로 돌아가기
        driver.switch_to.window(driver.window_handles[0])

    # 다음 페이지로 이동
    next_page_url = f'https://www.boannews.com/media/t_list.asp?Page={page+1}&kind=0'
    driver.get(next_page_url)
    time.sleep(2)  # 2초 대기 (페이지 로딩을 위해)

# 웹 드라이버 종료
driver.quit()


# CSV 파일로 저장
with open('news_data3.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Date', 'Content'])
    writer.writerows(news_data)

print('크롤링이 완료되었습니다.')